In [ ]:
# Core libraries
# pip install numpy==1.26.4
# pip install opencv-python==4.8.1.78
# pip install tqdm==4.66.1
# pip install moviepy==2.0.0.post5


In [ ]:
import base64
import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:

def region_selection(image):
    mask = np.zeros_like(image)
    if len(image.shape) > 2:
        ignore_mask_color = (255,) * image.shape[2]
    else:
        ignore_mask_color = 255

    rows, cols = image.shape[:2]
    bottom_left  = [int(cols * 0.1), int(rows * 0.95)]
    top_left     = [int(cols * 0.4), int(rows * 0.6)]
    bottom_right = [int(cols * 0.9), int(rows * 0.95)]
    top_right    = [int(cols * 0.6), int(rows * 0.6)]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    return cv2.bitwise_and(image, mask)

def hough_transform(image):
    return cv2.HoughLinesP(image, 1, np.pi/180, 20, np.array([]), 20, 500)

def average_slope_intercept(lines):
    left_lines, right_lines = [], []
    left_weights, right_weights = [], []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1==x2: continue
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2 + (x2-x1)**2)
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append(length)
            else:
                right_lines.append((slope, intercept))
                right_weights.append(length)
    left_lane = np.dot(left_weights, left_lines)/np.sum(left_weights) if left_weights else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if right_weights else None
    return left_lane, right_lane

def pixel_points(y1,y2,line):
    if line is None: return None
    slope, intercept = line
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return ((x1,int(y1)),(x2,int(y2)))

def lane_lines(image, lines):
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = int(y1*0.6)
    return pixel_points(y1,y2,left_lane), pixel_points(y1,y2,right_lane)

def draw_lane_lines(image, lines, color=[255,0,0], thickness=12):
    line_image = np.zeros_like(image)
    for line in lines:
        if line: cv2.line(line_image,*line,color,thickness)
    return cv2.addWeighted(image,1.0,line_image,1.0,0.0)

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    edges = cv2.Canny(blur,50,150)
    region = region_selection(edges)
    hough = hough_transform(region)
    if hough is None:
        return frame
    return draw_lane_lines(frame,lane_lines(frame,hough))


def process_video_opencv(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps==0: fps=24  # default if FPS not available
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    pbar = tqdm(total=total_frames, desc="Processing video", ncols=80)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed = process_frame(frame)
        out.write(processed)
        # pbar.update(1)

    cap.release()
    out.release()
    pbar.close()
    print("✅ Video saved:", output_path)

def play_video_cv2(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    while True:
        ret, frame = cap.read()
        if not ret: break
        cv2.imshow("Lane Detection", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

input_video = "test2.mp4"
output_video = "output.mp4"

process_video_opencv(input_video, output_video)


In [14]:
process_video_opencv(input_video, output_video)
# process_video(input_video, output_video)

Processing video:   0%|                                | 0/1295 [00:25<?, ?it/s]

✅ Video saved: output.mp4


In [16]:
play_video_cv2(output_video)
# display_video_notebook(output_video)

In [23]:
# Core libraries
# !pip install numpy==1.25.2
# !pip install pandas==2.1.1

# OpenCV for image/video processing
# !pip install opencv-python==4.8.1.78
# !pip install opencv-python-headless==4.8.1.78  # Colab-safe

# MoviePy for video editing
# !pip install moviepy==1.0.3

# tqdm for progress bars
# !pip install tqdm==4.66.1

# IPython display for inline video
# !pip install ipython==8.17.0

